In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import yfinance as yf
import math

In [2]:
def procces_data_for_labels(ticker_df, ticker):
    hm_days = 7
    ticker_df.fillna(0)
    ticker_df.replace(0,0.001)
    for i in range (1, hm_days+1):
        ticker_df['{}_{}d'.format(ticker, i)] =  (ticker_df["Close"].shift(-i) - ticker_df["Close"]) / ticker_df["Close"]  
    return ticker_df

In [3]:
def in_hold_sell_for_svm(*args):
    cols = [ c for c in args ]
    #decision boundary to buy or sell  +1 -1  0 
    #1:buy 0:hold -1:sell
    #     could add moving avg  to logic new kind of algo. good for turkish market.
    #     more then 7day low && more than 200ma -> buy
    #     "$more_than_learneable_parameter_with_ml"% return sell.
    change_req = 0.03
    for colm in cols:
        if colm > change_req: return 1
        elif colm == change_req: return 0
        else : return -1

    

In [4]:
def extract_features(ticker_df, ticker):
    #guessings of next 7 day, creating labels/pre-processing input.
    ticker_df['target_{}'.format(ticker)] = list(map(in_hold_sell_for_svm, *[ticker_df['{}_{}d'.format(ticker, i)] for i in range(1, 8, 1)]  ) ) 
    df_vals = ticker_df.pct_change()
    
    df_vals = df_vals.replace([-np.inf,np.inf],np.nan)
    df_vals = df_vals.fillna(0.001)
    ticker_df['target_{}'.format(ticker)] = ticker_df['target_{}'.format(ticker)].replace( [-np.inf,np.inf], np.nan )
    ticker_df['target_{}'.format(ticker)] = ticker_df['target_{}'.format(ticker)].fillna(0.001)
    
    str_vals = [ str(val) for val in ticker_df['target_{}'.format(ticker)]  ]
    print("Data hyperplane/spread :", Counter(str_vals) )
    X = df_vals.values
    y = ticker_df['target_{}'.format(ticker)].values
    
    return X, y

In [5]:
from sklearn.model_selection import train_test_split
from sklearn import svm,neighbors
from sklearn.ensemble  import VotingClassifier, RandomForestClassifier 
from collections import Counter

In [6]:
def Svm_finance(ticker_df ,ticker):
    X, y = extract_features(ticker_df, ticker)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
    clf = neighbors.KNeighborsClassifier()
    clf.fit( X_train, y_train )
    conf = clf.score(X_test, y_test )
    predictions = clf.predict(X_test)
    print("Accuracy : ", conf)
    print('Predicted Spread:', Counter( predictions )  )
    return conf

In [9]:
tickers_important_local = ['PETKM.IS', 'THYAO.IS', 'ISCTR.IS', 'TUPRS.IS', 'SISE.IS']
#these stocks for turkish market are good example of investment because of consistent good returns in long term invesment.
# In low quality high momentum stock, accuracy would differ dramatically in a bad way. 
tickers_histo = {}
for ticker in tickers_important_local :
    tick = yf.Ticker(ticker).history(period = "10y")
    tick.dropna(inplace = True)
    tick = pd.DataFrame(tick["Close"])
    tick = procces_data_for_labels( tick, ticker)
    tickers_histo[ticker] = tick
    Svm_finance(tick ,ticker)
    
print(tickers_histo['PETKM.IS'].iloc[-20:-18])

Data hyperplane/spread : Counter({'-1': 2438, '1': 156})
Accuracy :  0.9614791987673343
Predicted Spread: Counter({-1: 620, 1: 29})
Data hyperplane/spread : Counter({'-1': 2381, '1': 213})
Accuracy :  0.9352850539291218
Predicted Spread: Counter({-1: 621, 1: 28})
Data hyperplane/spread : Counter({'-1': 2428, '1': 162})
Accuracy :  0.9737654320987654
Predicted Spread: Counter({-1: 625, 1: 23})
Data hyperplane/spread : Counter({'-1': 2438, '1': 156})
Accuracy :  0.9553158705701078
Predicted Spread: Counter({-1: 630, 1: 19})
Data hyperplane/spread : Counter({'-1': 2394, '1': 200})
Accuracy :  0.9645608628659477
Predicted Spread: Counter({-1: 620, 1: 29})
            Close  PETKM.IS_1d  PETKM.IS_2d  PETKM.IS_3d  PETKM.IS_4d  \
Date                                                                    
2020-11-30   4.21     0.014252     0.007126     0.011877     0.099762   
2020-12-01   4.27    -0.007026    -0.002342     0.084309     0.187354   

            PETKM.IS_5d  PETKM.IS_6d  PETKM.IS_

In [8]:
"""
moving avarages implementation
for key,value in tickers_histo.items():
    value['100ma']=value['Close'].rolling(window =100).mean()
    value.dropna(inplace=True)
"""


"\nfor key,value in tickers_histo.items():\n    value['100ma']=value['Close'].rolling(window =100).mean()\n    value.dropna(inplace=True)\n"